In [ ]:
from dotenv import load_dotenv
load_dotenv()
import os
ds_api_key = os.getenv("DEEP_SEEK_API_KEY")
openai_api_key = os.getenv("OPENAI_API_KEY")

In [ ]:
from langchain_deepseek import ChatDeepSeek
from langchain_openai import ChatOpenAI
deepseek = ChatDeepSeek(
    api_key=ds_api_key,
    model="deepseek-chat"
)

openai = ChatOpenAI(
    api_key=openai_api_key,
    base_url="https://yunwu.ai/v1",
    model_name="gpt-5-mini",
)

In [ ]:
from langchain_core.tools import tool
import requests
import yaml
from pathlib import Path
from typing import List, Dict, Any

def web_search_tool_kit():
    # 建议将 URL 外部化或作为配置
    searx_url = "https://sousuo.emoe.top/search"
    crawl4ai_url = "http://10.1.2.4:11235/md"

    @tool(parse_docstring=True)
    def web_search(query: str, max_results: int = 5, categories: str = "general", language: str ="zh-CN", engine: str | None = None) -> str:
        """
        利用 SearXNG 引擎进行互联网搜索。适用于获取实时新闻、技术文档或百科知识。

        Args:
            query (str): 具体的搜索关键词。
            max_results (int): 期望返回的结果条数，默认为 5。
            categories (str): 搜索类别。可选值: 'general', 'it', 'science', 'news', 'images', 'videos'。
            language (str): 搜索语言。默认为 "zh-CN"。
            engine (str | None): 指定使用的搜索引擎名称，如 "google", "bing" 等。默认为 None，表示使用默认引擎。

        Returns:
            str: 格式化的搜索结果列表，每条包含标题、来源链接和内容摘要。
        """
        params = {
            "q": query,
            "format": "json",
            "engine": engine,
            "categories": categories,
            "language": language,
        } if engine else {
            "q": query,
            "format": "json",
            "categories": categories,
            "language": language,
        }

        try:
            response = requests.get(searx_url, params=params, timeout=15)
            response.raise_for_status()
            raw_results = response.json().get("results", [])
        except Exception as e:
            return f"搜索失败: {str(e)}"

        # --- 核心优化：数据清洗 ---
        processed_results = []
        # 只取前 max_results 条，避免 Token 溢出
        for res in raw_results[:max_results]:
            # 提取 AI 需要的关键信息
            title = res.get("title", "无标题")
            link = res.get("url", "无链接")
            snippet = res.get("content", "无描述")

            # 格式化为易于 AI 阅读的字符串
            processed_results.append(f"标题: {title}\n链接: {link}\n摘要: {snippet}\n---")

        if not processed_results:
            return "未找到相关结果。"

        return "\n".join(processed_results)


    @tool(parse_docstring=True)
    def web_reader(url: str, priority: int = 1) -> str:
        """
        当你需要阅读特定网页的详细内容时使用此工具。
        支持动态加载的网页（如单页应用）。

        Args:
            url (str): 要读取的完整网页 URL。
            priority (int): 爬取优先级，1-10 之间。

        Returns:
            str: 网页的正文内容（Markdown 格式）。
        """
        payload = {
            "url": url,
            "f": "fit"
        }

        try:
            response = requests.post(crawl4ai_url, json=payload, timeout=30)
            response.raise_for_status()

            data = response.json()

            if data.get("success") and data.get("markdown"):
                content = data.get("markdown", "")
                if len(content) > 5000:
                    return content[:5000] + "\n\n(内容过长，已自动截断...)"
                return content
            else:
                return f"未能提取内容: {data.get('error', '未知错误')}"

        except Exception as e:
            return f"读取网页失败: {str(e)}"

    return [web_search, web_reader]

def commomon_tool_kit():

    @tool
    def now():
        """获取当前的日期和时间。"""
        from datetime import datetime
        return datetime.now().strftime("%Y-%m-%d %H:%M:%S")

    return [now]

def prompt_tool_kit(model=None):
    """
    创建 Meta Prompt 工具包，用于运行提示词工程工作流。

    Args:
        model: LangChain Chat Model (如 ChatDeepSeek, ChatOpenAI)。如果为 None，使用全局 deepseek。

    Returns:
        List[Tool]: 包含四个核心工具的列表
    """
    # 使用全局 deepseek 模型作为默认
    if model is None:
        from langchain_deepseek import ChatDeepSeek
        model = deepseek

    meta_prompts_dir = Path("meta_prompts")

    def load_prompt_template(name: str) -> Dict[str, Any]:
        """加载 YAML 格式的 prompt 模板"""
        yaml_path = meta_prompts_dir / f"{name}.yaml"
        with open(yaml_path, 'r', encoding='utf-8') as f:
            return yaml.safe_load(f)

    def render_messages(template: Dict, **kwargs) -> List[Dict[str, str]]:
        """渲染消息模板，替换变量占位符"""
        messages = []
        for msg in template['messages']:
            content = msg['content']
            # 替换 {{variable}} 格式的占位符
            for key, value in kwargs.items():
                content = content.replace(f"{{{{{key}}}}}", str(value))
            messages.append({"role": msg['role'], "content": content})
        return messages

    def call_llm(messages: List[Dict[str, str]]) -> str:
        """调用 LLM 并返回响应"""
        from langchain_core.messages import HumanMessage, SystemMessage

        lc_messages = []
        for msg in messages:
            if msg['role'] == 'system':
                lc_messages.append(SystemMessage(content=msg['content']))
            elif msg['role'] == 'user':
                lc_messages.append(HumanMessage(content=msg['content']))
            elif msg['role'] == 'assistant':
                # 历史对话中的 assistant 消息转换为 HumanMessage
                lc_messages.append(HumanMessage(content=f"(Previous assistant response): {msg['content']}"))

        response = model.invoke(lc_messages)
        return response.content

    # ========== 工具 1: Prompt Architect ==========
    @tool(parse_docstring=True)
    def prompt_architect(requirement: str) -> str:
        """
        将用户需求转换为精确的技术规格文档 (JSON)。

        Args:
            requirement (str): 用户的需求描述，支持中文或英文。

        Returns:
            str: JSON 格式的技术规格文档，包含 input/output schema、task、goal、constraint。
        """
        template = load_prompt_template('prompt_architect')
        messages = render_messages(template, requirement=requirement)
        return call_llm(messages)

    # ========== 工具 2: Data Generator ==========
    @tool(parse_docstring=True)
    def data_generator(
        num: int,
        analysis: str,
        notion: str = "Generate diverse test cases covering edge cases",
        require_output: bool = True
    ) -> str:
        """
        基于技术规格生成高质量的合成测试数据集。

        Args:
            num (int): 生成的测试用例数量。
            analysis (str): Prompt Architect 生成的技术规格 (JSON)。
            notion (str): 特定指令/关注点，如 "测试边界条件" 或 "测试多语言支持"。
            require_output (bool): 是否生成预期输出，默认 True。

        Returns:
            str: JSON 格式的数据集，包含 dataset 键和测试用例列表。
        """
        template = load_prompt_template('data_generator')
        messages = render_messages(
            template,
            num=num,
            analysis=analysis,
            notion=notion,
            require_output=str(require_output).lower()
        )
        return call_llm(messages)

    # ========== 工具 3: Prompt Builder ==========
    @tool(parse_docstring=True)
    def prompt_builder(analysis: str, test_data: str) -> str:
        """
        将技术规格和测试数据转换为可直接调用的 messages 列表 (JSON)。

        Args:
            analysis (str): Prompt Architect 生成的技术规格 (JSON)。
            test_data (str): Data Generator 生成的测试数据集 (JSON)。

        Returns:
            str: JSON 数组，包含完整的 messages 列表，可直接用于 API 调用。
        """
        template = load_prompt_template('prompt_builder')
        messages = render_messages(
            template,
            analysis=analysis,
            test_data=test_data
        )
        return call_llm(messages)

    # ========== 工具 4: Prompt Evaluator ==========
    @tool(parse_docstring=True)
    def prompt_evaluator(
        analysis: str,
        input_data: str,
        actual_output: str,
        expected_output: str = "None provided"
    ) -> str:
        """
        评估 AI Agent 的执行结果，返回评分和改进建议。

        Args:
            analysis (str): 技术规格文档 (JSON)，包含目标和约束。
            input_data (str): 输入给 Agent 的数据。
            actual_output (str): Agent 实际生成的输出。
            expected_output (str): 预期的正确答案（可选）。

        Returns:
            str: JSON 格式的评估报告，包含 reasoning、issues、suggestions、score (0-100)。
        """
        template = load_prompt_template('prompt_evaluator')
        messages = render_messages(
            template,
            analysis=analysis,
            input_data=input_data,
            expected_output=expected_output,
            actual_output=actual_output
        )
        return call_llm(messages)

    return [prompt_architect, data_generator, prompt_builder]

In [ ]:
from deepagents import create_deep_agent
from langfuse.langchain import CallbackHandler
ds_agent = create_deep_agent(
    name="deepseek-agent",
    model=deepseek,
    tools=prompt_tool_kit(deepseek) + web_search_tool_kit(),
    system_prompt="你是一个提示词生成专家,有一系列工具供你使用,对于用户提出的需求,你无需再和用户交流,全自动借用工具完成任务, 考虑优先使用提示词工程工具包中的工具, 理想流程为先使用Prompt Architect生成技术规格,再使用Data Generator生成测试数据,然后使用Prompt Builder生成最终提示词",
)


In [ ]:
ds_agent.invoke({
    "messages" : [
        {"role": "user", "content": "我需要一个提示词,他的功能是帮助用户准确翻译英文博客,输入是经过处理的markdown英文文本,输出是翻译后的中文markdown文本"}]
},
config={"callbacks": [CallbackHandler()]}
)

In [ ]:
from deepagents.backends import CompositeBackend, StateBackend, StoreBackend, FilesystemBackend
from langgraph.store.memory import InMemoryStore
from deepagents import create_deep_agent
from langfuse.langchain import CallbackHandler
composite_backend = lambda rt: CompositeBackend(
    default=StateBackend(rt),
    routes={
        "/memories/": FilesystemBackend(root_dir="/home/zhonghan.chen/code/promptx/reports", virtual_mode=True),
    }
)
search_agent = create_deep_agent(
    name="search-agent",
    model=deepseek,
    backend=composite_backend,
    store=InMemoryStore(),
    tools=web_search_tool_kit() + commomon_tool_kit(),
    system_prompt="你是一个互联网搜索专家,有一系列工具供你使用,对于用户提出的需求,你无需再和用户交流,全自动借用工具完成任务,当你认为任务完成时,将报告写入/memories/xxx_report.md",
)

search_agent.invoke({
    "messages" : [
        {"role": "user", "content": "请帮我搜索关于最近clawd bot技术发展的新闻,并生成一份包含主要内容的报告"}]
    },
config={"callbacks": [CallbackHandler()]}
)